In [1]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import os

In [3]:
def calculateBioHeat(axial_loc,radial_loc,trans,medium,heat,pressure2D,z_values,r_values,iscomplete=0):
    """
    INPUT ARG
        axial_loc == [m] Coordinate Z of Interest Location
        radial_loc == [m] Coordinate R of Interest Location

        trans == dictionary of transducer properties
            "freq" == [Hz] Transmit Frequency of transducer
            "radius" == [m] Radius of transducer probe
            "focus" == [m] Focal point in space 
            "initPressure" == [Pa] Initial Pressure output by transducer

        medium == calls medium properties
            "speed" == [m/s] Speed of Sound
            "density" == [kg/m^3] Density 
            "absCoeff" == [Np/(m*MHz^2)] Absorption Coefficient
            "specHeatCap" == [J/(kg*K)] Specific Heat Capacity
            "thermDiff" == [(m^2)/s] Thermal Diffusivity 

        heat == calls heating parameters
            "numTime" == [s]
            "HeatTime" == [s]
            "CoolTime" == [s]
            "DutyCycle" == [%] 
        
    OUTPUT ARG
        time_axis == axis in time
        temp_vec == Temperature Vector over Time
        Q == Heat Map
    """


    # Edit and Transform Transducer Properties
    d = trans["focus"]
    abs_Coeff = medium["absCoeff"] * (pow((trans["freq"]/(1e6)),2)) # POWER LAW
    dutyCycleRatio = heat["DutyCycle"]/100

    # Set Axes
    axial_min = 0.001
    axial_max = 2*d
    radial_min = -trans["radius"]
    dz = (axial_max - axial_min) / (len(z_values)-1)
    print(dz)
    dr = -1 * radial_min / ((len(r_values)-1)/2)
    print(dr)
    temp_dist_times = [30]
    dt = heat["numTime"]
    time_axis = np.arange(0,(heat["HeatTime"]+heat["CoolTime"]+dt),dt)


    # Bioheat
    intensity2D = pow(pressure2D,2) * pow(trans["initPressure"],2) * dutyCycleRatio / (2 * medium["density"] * medium["speed"])
    Q = intensity2D * 2 * abs_Coeff / (medium["density"] * medium["specHeatCap"])

    # Temperature Matrix
    num_dist = 0
    numR,numZ = np.shape(pressure2D)
    temp_dist = np.zeros((numR,numZ),dtype=np.float64)

    r_center_idx = round(((0-radial_min)/dr)+1)
    radial_temp_idx = round(((radial_loc-radial_min)/dr)+1)
    axial_temp_idx = round(((axial_loc-axial_min)/dz)+1)
    temp_vec = []
    temp_vec.append(temp_dist[axial_temp_idx,radial_temp_idx])

    heat_dist_indices = []
    cool_dist_indices = []
    for ii in range(1,len(temp_dist_times)):
        time_value = temp_dist_times[ii]
        if time_value >= 0 and time_value <= heat["HeatTime"]:
            heat_dist_indices = heat_dist_indices.append(round(time_value/dt))
        elif time_value > heat["HeatTime"]and time_value <= (heat["HeatTime"] + heat["CoolTime"]):
            cool_dist_indices = cool_dist_indices.append(round((time_value - heat["HeatTime"])/dt))
    

    # FTCS Scheme ADD FUNCTION FOR HEAT
    term1 = ((dt * medium["thermDiff"]) / pow(dr,2))
    for time_step in np.arange(dt,heat["HeatTime"]+dt,dt):
        curr_temp_dist = temp_dist
        for ii in range(1,numR-1):
            for jj in range(1, numZ-1):
                currTemp = curr_temp_dist[ii,jj]

                term2z = (curr_temp_dist[ii+1,jj] - (2 * curr_temp_dist[ii,jj]) + curr_temp_dist[ii-1,jj])
                term2r = (curr_temp_dist[ii,jj+1] - (2 * curr_temp_dist[ii,jj]) + curr_temp_dist[ii,jj-1])
                term3r = (1/r_values[jj]) * (curr_temp_dist[ii,jj+1] - curr_temp_dist[ii,jj-1]) * ((dt * medium["thermDiff"]) / (2*dr)) # OVERFLOW ERROR
                
                z_component = term1 * term2z
                r_component = term1 * term2r + term3r

                if jj == r_center_idx:
                    r_component = 4*term1*(curr_temp_dist[ii,jj+1] - curr_temp_dist[ii,jj])

                Q_US = dt * Q[ii,jj]
                temp_dist[ii,jj] = currTemp + z_component + r_component + Q_US

        print(time_step)
        temp_vec.append(temp_dist[axial_temp_idx,radial_temp_idx])

    print('COOLING')

    # FTCS Scheme ADD FUNCTION FOR COOL
    for time_step in np.arange(dt,heat["CoolTime"],dt):
        curr_temp_dist = temp_dist
        for ii in range(1,numR-2):
            for jj in range(1, numZ-2):
                currTemp = curr_temp_dist[ii,jj]

                z_component = ((dt * medium["thermDiff"]) / pow(dr,2)) \
                * (curr_temp_dist[ii+1,jj] - (2 * curr_temp_dist[ii,jj]) + curr_temp_dist[ii-1,jj]) 

                r_component = ((dt * medium["thermDiff"]) / pow(dr,2)) \
                * (curr_temp_dist[ii,jj+1] - (2 * curr_temp_dist[ii,jj]) + curr_temp_dist[ii,jj-1]) \
                + (1/r_values[jj]) * (curr_temp_dist[ii,jj+1] - curr_temp_dist[ii,jj-1]) \
                * (dt * medium["thermDiff"] / (2*dr))

                if jj == r_center_idx:
                    r_component = 4*((dt * medium["thermDiff"]) / pow(dr,2)) \
                    * (curr_temp_dist[ii,jj+1] - curr_temp_dist[ii,jj])

                temp_dist[ii,jj] = currTemp + z_component + r_component
        print(time_step)
        temp_vec.append(temp_dist[axial_temp_idx,radial_temp_idx])


    print("Bioheat Calculation Complete")
    iscomplete = 1
    return time_axis, temp_vec, Q, iscomplete


In [7]:
filename1 = "df_pressure2D_placeholder.csv"
filename2 = "z_axis_placeholder.csv"
filename3 = "r_axis_placeholder.csv"

# Get Files   
directory = os.getcwd()  
df_pressure2D = np.array(pd.read_csv((directory + "\\" +  filename1)))[:,1:]
z_axis = np.array(pd.read_csv((directory + "\\" +  filename2)))[:,1]
r_axis = np.array(pd.read_csv((directory + "\\" +  filename3)))[:,1]

# Set Input Parameters
trans = settransducer.setTransducer(1*1e6,0.02,0.05,1*1e6) # freq (MHz --> Hz), radius, focus, init pressure (MPa --> Pa)
medium = setmedium.setMedium('Water', dict())
heat = dict(numTime = 5e-3, HeatTime = float(30), CoolTime = float(30), DutyCycle = int(100))

observeZ = 0.05 # [m]
observeR = 0

# Bioheat Equation for Location in Space
time_axis, temp_vec, Q, iscomplete = calculateBioHeat(observeZ,observeR,trans,medium,heat,df_pressure2D,z_axis,r_axis,iscomplete=0)



0.00099
0.0004
0.005
0.01
0.015
0.02
0.025
0.030000000000000002
0.034999999999999996
0.04
0.045
0.049999999999999996
0.055
0.06
0.065
0.07
0.07500000000000001
0.08
0.085
0.09000000000000001
0.095
0.1
0.10500000000000001
0.11
0.115
0.12000000000000001
0.125
0.13
0.135
0.14
0.14500000000000002
0.15
0.155
0.16
0.165
0.17
0.17500000000000002
0.18000000000000002
0.185
0.19


C:\Users\geral\AppData\Local\Temp\ipykernel_3736\2253505518.py:86: RuntimeWarning:

overflow encountered in scalar multiply

C:\Users\geral\AppData\Local\Temp\ipykernel_3736\2253505518.py:89: RuntimeWarning:

invalid value encountered in scalar add

C:\Users\geral\AppData\Local\Temp\ipykernel_3736\2253505518.py:84: RuntimeWarning:

invalid value encountered in scalar add

C:\Users\geral\AppData\Local\Temp\ipykernel_3736\2253505518.py:84: RuntimeWarning:

invalid value encountered in scalar subtract



0.195
0.2
0.20500000000000002
0.21000000000000002
0.215
0.22
0.225
0.23
0.23500000000000001
0.24000000000000002
0.245
0.25
0.255
0.26
0.265
0.27
0.275
0.28
0.28500000000000003
0.29000000000000004
0.295
0.3
0.305
0.31
0.315
0.32
0.325
0.33
0.335
0.34
0.34500000000000003
0.35000000000000003
0.35500000000000004
0.36
0.365
0.37
0.375
0.38
0.385
0.39
0.395
0.4
0.405
0.41000000000000003
0.41500000000000004
0.42000000000000004
0.425
0.43
0.435
0.44
0.445
0.45
0.455
0.46
0.465
0.47000000000000003
0.47500000000000003
0.48000000000000004
0.485
0.49
0.495
0.5
0.505
0.51
0.515
0.52
0.525
0.53
0.535
0.54
0.545
0.55
0.555
0.56
0.5650000000000001
0.5700000000000001
0.5750000000000001
0.5800000000000001
0.585
0.59
0.595
0.6
0.605
0.61
0.615
0.62
0.625
0.63
0.635
0.64
0.645
0.65
0.655
0.66
0.665
0.67
0.675
0.68
0.685
0.6900000000000001
0.6950000000000001
0.7000000000000001
0.7050000000000001
0.71
0.715
0.72
0.725
0.73
0.735
0.74
0.745
0.75
0.755
0.76
0.765
0.77
0.775
0.78
0.785
0.79
0.795
0.8
0.805
0.8